## Notebook 5 – Oppsummering og visuelle innsikter  

**LuftDataQC: PM2.5-luftkvalitet i Oslo (2023)**  
Datakilde: [https://api.nilu.no](https://api.nilu.no)  

---

### Prosjektoversikt  
**Mål:** Presentere en strukturert og visuell oppsummering av hele LuftDataQC-pipelinen, med integrerte resultater fra datainnsamling, utforskende analyse og anomalideteksjon.  
**Metode:** Oppsummere og visualisere funn fra Notebook 1-3 gjennom visuell historiefortelling, samt fremheve sentrale innsikter og metodiske begrensninger.  
**Verktøy (hele pipelinen):** Python (`requests`, `json`, `pathlib`, `sqlite3`, `pandas`, `numpy`,  
`matplotlib`, `seaborn`, `scikit-learn`, `plotly`, `pyarrow`), Jupyter Notebook  

---

### Reproduserbarhet – hurtigreferanse  

**Inndata (fra tidligere notebooken):**  
- `data/processed/pm25_2023.sqlite`  
- Figurer generert i Notebook1-3  

**Utdata (denne notebooken):**  
- `04_report.ipynb` *(porteføljeoppsummering med figurer)*  

**Parametere (denne notebooken):**  
- **År:** 2023  
- **Stasjoner:** Skøyen, Furulund  
- **Omfang:** Sluttrapport og visuell historiefortelling  


## Introduksjon

Denne rapporten presenterer resultatene fra **LuftDataQC**-prosjektet, som analyserte timevise PM2.5-konsentrasjoner målt av Norsk institutt for luftforskning (NILU) i Oslo gjennom 2023. Prosjektet demonstrerer en komplett datavitenskapelig arbeidsflyt fra rådatainnhenting til anomalideteksjon – og fremhever praktiske ferdigheter innen **API-integrasjon, SQL-basert lagring, utforskende analyse, statistisk testing og maskinlæring**.

To NILU-stasjoner ble valgt ut for detaljert studie på grunn av deres kontrasterende miljøer:

* **Skøyen** — urban trafikkpåvirkning.  
* **Furulund** — boligområde i bakgrunnsmiljø.  

PM2.5 er en nøkkelkomponent i luftforurensning, assosiert med hjerte- og lungesykdom samt økt dødelighet, og utgjør derfor et viktig mål for transparent analyse og beslutningsstøtte.

---

## Datainnsamling og lagring (Notebook 1)

Timevise PM2.5-målinger for alle NILU-stasjoner i Oslo i 2023 ble hentet via NILUs åpne API og lagret i en **SQLite**-database for reproduserbarhet og effektiv spørring.

![PM2.5-stasjonsdekning – 2023](../results/pm25_station_coverage_2023.png)  
*Figur: PM2.5-stasjonsdekning (2023)* – [Vis interaktiv graf (HTML)](../results/pm25_station_coverage_2023.html)

---

## Utforskende dataanalyse (Notebook 2)

### Datakvalitet og fordeling

Kvalitetskontrollen inkluderte sjekk av manglende verdier, duplikater og rimelige måleintervaller. Fordelingsforskjeller mellom stasjoner ble vurdert for å underbygge valg av statistiske metoder.

![PM2.5-fordeling – Boksplott + histogram (Skøyen vs. Furulund)](../results/pm25_distribution_box_hist_skøyen_furulund.png)  
*Figur: PM2.5-fordeling – boksplott + histogram (Skøyen vs. Furulund)*  

**Begrunnelse for ikke-parametrisk testing:** PM2.5-fordelingene var skjeve og ikke-normale. Derfor ble **Mann–Whitney U-testen** valgt fremfor t-testen, grunnet robusthet mot uteliggere og fravær av normalitetskrav.  

### Timebaserte mønstre

Døgnvariasjonene reflekterer trafikk og aktivitetsnivå.

![PM2.5 – Timevise mønstre (Skøyen vs. Furulund)](../results/pm25_hourly_trends_skøyen_furulund.png)  
*Figur: PM2.5 – timevise mønstre (Skøyen vs. Furulund)*  

### Månedlige mønstre

Sesongvariasjonene viser forhøyede vinterverdier og lavere nivåer om sommeren, i tråd med meteorologiske forhold og utslippsmønstre.

![PM2.5 – Månedlige trender (Skøyen vs. Furulund)](../results/pm25_monthly_trends_skøyen_furulund.png)  
*Figur: PM2.5 – månedlige trender (Skøyen vs. Furulund)*  

### Ukedager vs. helg

Lavere konsentrasjoner i helger samsvarer med redusert trafikk og aktivitet.

![PM2.5 – Ukedag vs. helg (2023)](../results/pm25_weekday_vs_weekend.png)  
*Figur: Ukedag vs. helg (2023)*  

### Trend og glatting

Glidende gjennomsnitt fremhever underliggende trender utover kortsiktig variasjon.

![PM2.5 – Trend (glidende gjennomsnitt, Skøyen vs. Furulund)](../results/pm25_trend_rolling_mean_skøyen_furulund.png)  
*Figur: PM2.5 – trend (glidende gjennomsnitt, Skøyen vs. Furulund)*  

**Tolkning:** Skøyen viser konsekvent høyere nivåer og større variasjon enn Furulund, i tråd med sin urbane plassering. Mann–Whitney U-testen indikerte en statistisk signifikant forskjell mellom stasjonsfordelingene.
_____

### Feature Engineering (variabelkonstruksjon) og anomalideteksjon (Notebook 3)
 
Dette proof-of-concept viser hvordan maskinlæring kan støtte **sanntidsluftkvalitetsovervåking** ved å markere uvanlige timevise PM2.5-verdier for faglig vurdering. De avledede variovervåkningablene gir et konsist tidsmessig kontekstgrunnlag. Anomalier kan representere enten reelle forurensningsepisoder eller **sensoravvik**.

### Steg 1 – Validering av kontamineringsgrad  
IsolationForest ble evaluert med kontamineringsverdier på **0,5 %, 1 %, 2 % og 5 %** for å kalibrere modellens følsomhet.  
Visuell vurdering indikerte at en kontamineringsgrad på **1 %** ga den mest hensiktsmessige balansen mellom følsomhet for sjeldne forurensningstopper og kontroll av falske positive.

![PM2.5 – Sammenligning av kontamineringsgrad (Skøyen)](../results/pm25_skøyen_contamination_comparison.png)  
*Figur: Sammenligning av kontamineringsgrad – Skøyen*  

![PM2.5 – Sammenligning av kontamineringsgrad (Furulund)](../results/pm25_furulund_contamination_comparison.png)  
*Figur: Sammenligning av kontamineringsgrad – Furulund*  

### Steg 2 - Tidsbasert feature engineering  
Tidsvariabler ble avledet fra tidsstempelet, inkludert time på døgnet, ukedag og helgeflagg (med lesbare navn tilpasset visualisering).  
Disse variablene gir et minimalt, men meningsfullt tidsmessig rammeverk som bidrar til å skille **forventede variasjoner** (f.eks. rushtrafikk, sesongmønstre) fra **avvikende hendelser**.

### Resultater (1 % kontaminering)  
Med valgt parameter identifiserte IsolationForest kandidat-anomalier ved begge stasjoner. Visuell inspeksjon viser at noen anomalier samsvarer med sannsynlige forurensningstopper, mens andre kan være knyttet til sensorfeil.  
**Faglig ekspertise er avgjørende** for å tolke de markerte hendelsene i kontekst.

![PM2.5 – anomalideteksjon (Skøyen)](../results/pm25_skøyen_anomalies.png)  
*Figur: PM2.5 – anomalideteksjon (Skøyen)*  

![PM2.5 – anomalideteksjon (Furulund)](../results/pm25_furulund_anomalies.png)  
*Figur: PM2.5 – anomalideteksjon (Furulund)*  

---

## Innsikter

* **By–boligkontrast:** Skøyen registrerer høyere og mer varierende PM2.5 enn Furulund.  
* **Tidsstruktur:** Klare døgn- og sesongmønstre samsvarer med trafikkintensitet og vinterforhold.  
* **Maskinlæringsassistert (ML-assistert) - QA/QC (Kvalitetssikring / Kvalitetskontroll):** IsolationForest fremhever effektivt kandidat-anomalier for faglig vurdering.  
* **Statistisk belegg:** Ikke-parametriske tester bekrefter signifikante forskjeller mellom stasjonene.  

---

## Omfang og fremtidsperspektiver

* **Datagrunnlag:** Analysen fokuserte på NILU-validerte 2023-data. År 2024 ble utelatt da QA/QC - validering ikke var fullført på analysetidspunktet.  
* **Dekning:** Studien er avgrenset til to stasjoner i Oslo som en case-studie; pipelinen er imidlertid utformet for å kunne skalere til flere stasjoner og lengre tidsserier.  
* **Mulige utvidelser:**  
  - Integrere meteorologiske variabler (temperatur, vind, luftfuktighet) for bedre tolkning av anomalier.  
  - Utvide til flerårige og flerstasjonsanalyser for bredere anvendelse.  
  - Sammenligne alternative metoder for anomalideteksjon (f.eks. One-Class SVM, autoenkodere).  

---

## Konklusjon

Denne pilotanalysen demonstrerer muligheten for å anvende anomalideteksjon i miljøovervåkning.  
Den etablerte pipelinen – **datainnhenting - QC/EDA (Kvalitetskontroll/Utforskende dataanalyse) - feature engineering - anomalideteksjon - visualisering** - er reproduserbar og skalerbar.  
De markerte anomaliene utgjør **kandidater for faglig gjennomgang**, snarere enn endelige etiketter, noe som understreker behovet for kontekstuell tolkning før operativ bruk.


**Ende-til-ende-resultater:**  

- **Innhenting og lagring:** Hentet timevise PM2.5-data for 2023 fra NILUs åpne API og lagret i **SQLite** for sporbar og portabel analyse.  
- **Utforskning og testing:** Utførte QC og EDA (fordeling, time- og månedsmønstre, ukedag vs. helg, glidende trender); valgte **Mann–Whitney U** for ikke-normale fordelinger.  
- **Modellering:** Konstruerte tidsbaserte variabler og anvendte **IsolationForest** for å identifisere kandidat-anomalier til faglig vurdering.  
- **Visuell historiefortelling:** Samlede figurer fremhever kontraster mellom **Skøyen** (urbant) og **Furulund** (boligområde), og legger til rette for transparent tolkning.  

**Analytiske innsikter:**  
- **Kontrast:** Skøyen har generelt høyere og mer variabel PM2.5 enn Furulund på tvers av tidsdimensjoner.  
- **Tidsstruktur:** Døgn- og sesongmønstre samsvarer med menneskelig aktivitet og vinterforhold.  
- **Statistisk belegg:** Ikke-parametriske tester bekrefter signifikante forskjeller mellom stasjonene.  
- **ML-assistert QA/QC:** IsolationForest er effektiv for å screene uvanlige observasjoner før operativ bruk.  

**Reproduserbarhet og artefakter:**  
- Database: `data/processed/pm25_2023.sqlite`  
- Resultater (figurer): se **Figurindeks** nedenfor  
- Notebooks (pipeline): se **Navigasjonslenker** nedenfor  
- Parametere: År=2023; Stasjoner=Skøyen, Furulund (ingen hardkodede antakelser om dekning)  

---

## Referanser

- McDonald, J.H. (2014). *Handbook of Biological Statistics* (3. utg.). (Mann–Whitney U)  
- Ghasemi, A., og Zahediasl, S. (2012). Normality tests for statistical analysis: A guide for non-statisticians. **JCLA, 26(2)**, 68–72.  
- Wilcox, R.R. (2012). *Introduction to Robust Estimation and Hypothesis Testing* (3. utg.).  
- Liu, F.T., Ting, K.M., og Zhou, Z.H. (2008). Isolation Forest. *KDD*.  
- SciPy-dokumentasjon — `scipy.stats.mannwhitneyu`  
- scikit-learn-dokumentasjon — `sklearn.ensemble.IsolationForest`  

---

## Navigasjonslenker

- [Notebook 1 – Data Collection, Inspection and Storage](./01_data_sqlite.ipynb) 
- [Notebook 2 – Exploratory Analysis and Quality Checks](./02_exploratory_qc.ipynb)  
- [Notebook 3 - Feature Engineering and Anomaly Detection](./03_features_anomalies.ipynb)
- [Notebook 4 – Summary Report (EN)](./04_report.ipynb)  

---

## Figurindeks (filstier)

**Notebook 1**  

`../results/pm25_station_coverage_2023.png`  

**Notebook 2**  

`../results/pm25_distribution_box_hist_skøyen_furulund.png`  
`../results/pm25_hourly_trends_skøyen_furulund.png`  
`../results/pm25_monthly_trends_skøyen_furulund.png`  
`../results/pm25_weekday_vs_weekend.png`  
`../results/pm25_trend_rolling_mean_skøyen_furulund.png`  

**Notebook 3**  

`../results/pm25_skøyen_contamination_comparison.png`  
`../results/pm25_furulund_contamination_comparison.png`  
`../results/pm25_skøyen_anomalies.png`  
`../results/pm25_furulund_anomalies.png`  
`../results/pm25_skøyen_with_anomalies.csv`  
`../results/pm25_furulund_with_anomalies.csv`  
